# Storing Data

In [149]:
import os
from urllib.request import urlretrieve, urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import sqlite3
import csv

import random
import re
from random import shuffle

In [3]:
html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
imageLocation = bs.find('img', {'alt': 'python-logo'})['src']
urlretrieve (imageLocation, 'logo.jpg')

('logo.jpg', <http.client.HTTPMessage at 0x1079dc5f0>)

In [4]:
def getAbsoluteURL(source, baseUrl):
    if urlparse(baseUrl).netloc == '':
        return baseUrl + source
    return source

def getDownloadPath(fileUrl, downloadDir):
    parsed = urlparse(fileUrl)
    netloc = parsed.netloc.strip('/')
    path = parsed.path.strip('/')
    localfile = f'{downloadDir}/{netloc}/{path}'
    
    # Remove the filename from the path in order to 
    # make the directory structure leading up to it
    localpath = '/'.join(localfile.split('/')[:-1])
    if not os.path.exists(localpath):
        os.makedirs(localpath)
    return localfile

In [5]:
downloadDir = 'downloaded'
baseUrl = 'https://pythonscraping.com/'
baseNetloc = urlparse(baseUrl).netloc

html = urlopen(baseUrl)
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src=True)

for download in downloadList:
    fileUrl = getAbsoluteURL(download['src'], baseUrl)
    if fileUrl is not None:
        try:
            urlretrieve(fileUrl, getDownloadPath(fileUrl, downloadDir))
            print(fileUrl)
        except Exception as e:
            print(f'Could not retrieve {fileUrl} Error: {e}')

https://pythonscraping.com/wp-includes/js/jquery/jquery.min.js?ver=3.7.1
https://pythonscraping.com/wp-includes/js/jquery/jquery-migrate.min.js?ver=3.4.1
https://pythonscraping.com/wp-content/plugins/pagelayer/js/combined.js?ver=1.7.5
https://pythonscraping.com/wp-content/plugins/email-capture-lead-generation//js/eclg-public.js?ver=1.0.1
https://www.googletagmanager.com/gtag/js?id=GT-TNFZZK6
https://pythonscraping.com/wp-content/uploads/2023/04/python-logo-e1681354047443.png
https://pythonscraping.com/wp-content/uploads/2021/08/home1.jpg
https://pythonscraping.com/wp-content/uploads/2021/08/logo01-e1681353135199.png
https://pythonscraping.com/wp-content/plugins/email-capture-lead-generation//images/ajax_loader.gif
https://pythonscraping.com/wp-content/plugins/contact-form-7/includes/swv/js/index.js?ver=5.7.7
https://pythonscraping.com/wp-content/plugins/contact-form-7/includes/js/index.js?ver=5.7.7
https://pythonscraping.com/wp-content/themes/popularfx/js/navigation.js?ver=1.2.0


In [7]:
csvFile = open('test.csv', 'w+')
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number plus 2', 'number times 2'))
    for i in range(10):
        writer.writerow( (i, i+2, i*2))
finally:
    csvFile.close()

In [8]:
html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
# The main comparison table is currently the first table on the page
table = bs.find('table',{'class':'wikitable'})
rows = table.findAll('tr')
csvFile = open('editors.csv', 'wt+')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text().strip())
        writer.writerow(csvRow)
finally:
    csvFile.close()

In [140]:
# taken from GEEKS for GEEKS
# Connecting to sqlite
# connection object
db = sqlite3.connect('storing_data.sqlite')
 
# cursor object
cursor = db.cursor()
 
# Drop the table if already exists.
cursor.execute("DROP TABLE IF EXISTS pages")
 
# Creating table
# note primary key automatically auto increments
table = """ CREATE TABLE pages (
            id INTEGER PRIMARY KEY,
            title NVARCHAR(200),
            content NVARCHAR(10000),
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        ); """
 
cursor.execute(table)
 
print("Table is Ready")
 
# Close the connection
db.close()

Table is Ready


In [148]:
def store(title, content, cursor):
    # NOTE: the replace " is needed because otherwise it will breake it (since it's in double quotes inside the query)
    content = content.replace('"', '""')
    cursor.execute(f'INSERT INTO pages (title, content) VALUES ("{title}", "{content}")')
    cursor.connection.commit()

def getLinks(articleUrl, cursor):
    html = urlopen('http://en.wikipedia.org'+articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content, cursor)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))


In [142]:
db = sqlite3.connect('storing_data.sqlite')
cursor = db.cursor()

links = getLinks('/wiki/Kevin_Bacon', cursor)
try:
    while len(links) > 0:
         newArticle = links[random.randint(0, len(links)-1)].attrs['href']
         print(newArticle)
         links = getLinks(newArticle, cursor)
finally:
    db.close()

/wiki/Ghent_International_Film_Festival
/wiki/Gaston_Kabor%C3%A9
/wiki/IMDb_(identifier)
/wiki/Vince_Chhabria
/wiki/James_A._Soto
/wiki/Cathy_Ann_Bencivengo
/wiki/James_A._Soto
/wiki/Miranda_Du
/wiki/University_of_California,_Davis
/wiki/US_Club_Soccer
/wiki/MLS_Next
/wiki/Sacramento_Republic_FC
/wiki/ESPN
/wiki/Midland_Daily_News
/wiki/WLWT
/wiki/Comet_(TV_network)
/wiki/The_Ray_Bradbury_Theater
/wiki/The_Casual_Vacancy_(miniseries)
/wiki/Mind_Over_Murder_(TV_series)
/wiki/The_Baby_(TV_series)
/wiki/The_Chris_Rock_Show
/wiki/Wynton_Marsalis
/wiki/Ben_Stein
/wiki/Pseudoscience
/wiki/Scientometrics
/wiki/ArXiv_(identifier)
/wiki/Doi_(identifier)


KeyboardInterrupt: 

In [154]:
with sqlite3.connect('storing_data.sqlite') as db:
    print(db.execute('SELECT * FROM pages').fetchall())

[(1, 'Kevin Bacon', '\n', '2024-04-17 01:12:17'), (2, 'Film Fest Gent', '\n', '2024-04-17 01:12:18'), (3, 'Gaston Kaboré', "Gaston Kaboré (born 1951) is a Burkinabé film director and an important figure in Burkina Faso's film industry.[1] He has won awards for his films Wend Kuuni and Buud Yam. He is the founder of Imagine Institute, a school he opened in Ouagadougou, Burkina Faso in 2003, that provides workshops and residencies for film and television professionals.\n", '2024-04-17 01:12:18'), (4, 'IMDb', '\n', '2024-04-17 01:12:19'), (5, 'Vince Chhabria', "Vince Girdhari Chhabria (born November 27, 1969) is a United States district judge of the United States District Court for the Northern District of California and formerly a deputy city attorney at the San Francisco City Attorney's Office.\n", '2024-04-17 01:12:19'), (6, 'James A. Soto', 'James Alan Soto (born July 1, 1950) is  a United States district judge of the United States District Court for the District of Arizona and former

In [145]:
# CODE FROM AUTHOR, using pymysql
# import pymysql

# conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
#                        user='root', passwd='password', db='mysql', charset='utf8')
# cur = conn.cursor()
# cur.execute('USE scraping')

# def store(title, content):
#     cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content))
#     cur.connection.commit()

# def getLinks(articleUrl):
#     html = urlopen('http://en.wikipedia.org'+articleUrl)
#     bs = BeautifulSoup(html, 'html.parser')
#     title = bs.find('h1').get_text()
#     content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
#     store(title, content)
#     return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

# links = getLinks('/wiki/Kevin_Bacon')
# try:
#     while len(links) > 0:
#          newArticle = links[random.randint(0, len(links)-1)].attrs['href']
#          print(newArticle)
#          links = getLinks(newArticle)
# finally:
#     cur.close()
#     conn.close()

## Better Database structure

In [175]:
# taken from GEEKS for GEEKS
# Connecting to sqlite
# connection object
db = sqlite3.connect('wikipedia.sqlite')
 
# cursor object
cursor = db.cursor()
 
# Drop the table if already exists.
cursor.execute("DROP TABLE IF EXISTS pages")
 
# Creating table
# note primary key automatically auto increments
table = """ CREATE TABLE pages (
            id INTEGER PRIMARY KEY,
            url VARCHAR(255),
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        ); """
 
cursor.execute(table)

table = """ CREATE TABLE links (
            id INTEGER PRIMARY KEY,
            fromPageId INTEGER NULL,
            toPageId INTEGER NULL,
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        ); """
 
cursor.execute(table)
 
print("Table is Ready")
 
# Close the connection
db.close()

Table is Ready


In [180]:
def insertPageIfNotExists(url, cursor):
    cursor.execute(f'SELECT id FROM pages WHERE url = "{url}" LIMIT 1')
    page = cursor.fetchone()
    if not page:
        cursor.execute(f'INSERT INTO pages (url) VALUES ("{url}")')
        cursor.connection.commit()
        return cursor.lastrowid
    else:
        return page[0]

def loadPages(cursor):
    cursor.execute('SELECT url FROM pages')
    return [row[0] for row in cursor.fetchall()]

def insertLink(fromPageId, toPageId, cursor):
    cursor.execute(f'SELECT EXISTS(SELECT 1 FROM links WHERE fromPageId = {int(fromPageId)} AND toPageId = {int(toPageId)})')
    if not cursor.fetchone()[0]:
        cursor.execute(f'INSERT INTO links (fromPageId, toPageId) VALUES ({int(fromPageId)}, {int(toPageId)})')
        cursor.connection.commit()

def pageHasLinks(pageId, cursor):
    cursor.execute(f'SELECT EXISTS(SELECT 1 FROM links WHERE fromPageId = {int(pageId)})')
    return cursor.fetchone()[0]


def getLinks(pageUrl, recursionLevel, pages, cursor):
    if recursionLevel > 4:
        return

    pageId = insertPageIfNotExists(pageUrl, cursor)
    html = urlopen(f'http://en.wikipedia.org{pageUrl}')
    bs = BeautifulSoup(html, 'html.parser')
    links = bs.findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    links = [link.attrs['href'] for link in links]

    for link in links:
        linkId = insertPageIfNotExists(link, cursor)
        insertLink(pageId, linkId, cursor)
        if not pageHasLinks(linkId, cursor):
            print(f'Getting {link}')
            pages.append(link)
            getLinks(link, recursionLevel+1, pages, cursor)
        else:
            print(f'Already fetched {link}')

In [183]:
# with sqlite3.connect('wikipedia.sqlite') as db:
#     print(db.execute('SELECT * FROM pages WHERE url = "/wiki/Kevin_Bacon"').fetchone())

In [182]:
db = sqlite3.connect('wikipedia.sqlite')
cursor = db.cursor()

getLinks('/wiki/Kevin_Bacon', 0, loadPages(cursor), cursor) 
db.close()

Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/Kevin_Bacon
Already fetched /wiki/Kevin_Bacon
Already fetched /wiki/Kevin_Bacon
Getting /wiki/Kevin_Bacon_(disambiguation)
Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/Kevin_Bacon_(disambiguation)
Already fetched /wiki/Kevin_Bacon_(disambiguation)
Already fetched /wiki/Kevin_Bacon_(disambiguation)
Already fetched /wiki/Kevin_Bacon
Getting /wiki/Kevin_Bacon_(producer)
Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/Kevin_Bacon_(producer)
Already fetched /wiki/Kevin_Bacon_(producer)
Already fetched /wiki/Kevin_Bacon_(producer)
Already fetched /wiki/Kevin_Bacon_(disambiguation)
Getting /wiki/Rotherham
Already fetched /wiki/Main_Page
Already fetched /wiki/Main_Page
Already fetched /wiki/Rotherham
Already fetched /wiki/Rotherham
Already fetched /wiki/Rotherham
Getting /wiki/Geographic_coordinate_system
Already fetched /wiki/Main

KeyboardInterrupt: 

In [186]:
with sqlite3.connect('wikipedia.sqlite') as db:
    print(db.execute('SELECT * FROM pages').fetchall())
    print(db.execute('SELECT * FROM links').fetchall())

[(1, '/wiki/Kevin_Bacon', '2024-04-17 02:06:29'), (2, '/wiki/Main_Page', '2024-04-17 02:06:30'), (3, '/wiki/Wikipedia', '2024-04-17 02:06:30'), (4, '/wiki/English_Wikipedia', '2024-04-17 02:06:31'), (5, '/wiki/Internet_encyclopedia', '2024-04-17 02:06:32'), (6, '/wiki/Online_encyclopedia', '2024-04-17 02:06:32'), (7, '/wiki/Encyclopedia', '2024-04-17 02:06:32'), (8, '/wiki/Internet', '2024-04-17 02:06:32'), (9, '/wiki/Encarta', '2024-04-17 02:06:32'), (10, '/wiki/Encyclop%C3%A6dia_Britannica', '2024-04-17 02:06:32'), (11, '/wiki/Encyclopedia.com', '2024-04-17 02:06:32'), (12, '/wiki/Project_Gutenberg', '2024-04-17 02:06:32'), (13, '/wiki/ASCII', '2024-04-17 02:06:32'), (14, '/wiki/Encyclop%C3%A6dia_Britannica_Eleventh_Edition', '2024-04-17 02:06:32'), (15, '/wiki/Digitization', '2024-04-17 02:06:32'), (16, '/wiki/Lord_Chamberlain', '2024-04-17 02:06:32'), (17, '/wiki/Luqman', '2024-04-17 02:06:32'), (18, '/wiki/CD-ROM', '2024-04-17 02:06:32'), (19, '/wiki/Copyright', '2024-04-17 02:06:

In [184]:
# LIKE BEFORE
# from urllib.request import urlopen
# from bs4 import BeautifulSoup
# import re
# import pymysql
# from random import shuffle

# conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
#                        user='root', passwd='password', db='mysql', charset='utf8')
# cur = conn.cursor()
# cur.execute('USE wikipedia')

# def insertPageIfNotExists(url):
#     cur.execute('SELECT id FROM pages WHERE url = %s LIMIT 1', (url))
#     page = cur.fetchone()
#     if not page:
#         cur.execute('INSERT INTO pages (url) VALUES (%s)', (url))
#         conn.commit()
#         return cur.lastrowid
#     else:
#         return page[0]

# def loadPages():
#     cur.execute('SELECT url FROM pages')
#     return [row[0] for row in cur.fetchall()]

# def insertLink(fromPageId, toPageId):
#     cur.execute('SELECT EXISTS(SELECT 1 FROM links WHERE fromPageId = %s AND toPageId = %s)',(int(fromPageId), int(toPageId)))
#     if not cur.fetchone()[0]:
#         cur.execute('INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)', 
#                     (int(fromPageId), int(toPageId)))
#         conn.commit()

# def pageHasLinks(pageId):
#     cur.execute('SELECT EXISTS(SELECT 1 FROM links WHERE fromPageId = %s)', (int(pageId)))
#     return cur.fetchone()[0]


# def getLinks(pageUrl, recursionLevel, pages):
#     if recursionLevel > 4:
#         return

#     pageId = insertPageIfNotExists(pageUrl)
#     html = urlopen(f'http://en.wikipedia.org{pageUrl}')
#     bs = BeautifulSoup(html, 'html.parser')
#     links = bs.findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))
#     links = [link.attrs['href'] for link in links]

#     for link in links:
#         linkId = insertPageIfNotExists(link)
#         insertLink(pageId, linkId)
#         if not pageHasLinks(linkId):
#             print(f'Getting {link}')
#             pages.append(link)
#             getLinks(link, recursionLevel+1, pages)
#         else:
#             print(f'Already fetched {link}')
        
        
# getLinks('/wiki/Kevin_Bacon', 0, loadPages()) 
# cur.close()
# conn.close()

NOTE: I don't have an email server, and I've played around before iwth outlook. So I won't play around with the stuff below.

In [ ]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('The body of the email is here')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'ryan@pythonscraping.com'
msg['To'] = 'webmaster@pythonscraping.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

In [1]:
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

def sendMail(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] ='christmas_alerts@pythonscraping.com'
    msg['To'] = 'ryan@pythonscraping.com'

    s = smtplib.SMTP('localhost')
    s.send_message(msg)
    s.quit()

bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
while(bs.find('a', {'id':'answer'}).attrs['title'] == 'NO'):
    print('It is not Christmas yet.')
    time.sleep(3600)
    bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
sendMail('It\'s Christmas!', 
         'According to https://isitchristmas.com/, it is Christmas!')

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
'